## _CSE 574 - Assignment 2 (Decision Trees)_

In [1]:
# Imports
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### data loading

In [2]:
data = load_iris()

In [3]:
# print the desctiption of the dataset given by sklearn 
description = data['DESCR']
print(description)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

### data-preprocessing

In [4]:
# restructure the data to form a dataframe for data pre-processing
features = data['data']
target = data['target'].reshape(-1,1)

feature_names = data['feature_names']
target_names = data['target_names']

dataset = np.hstack((features, target))

# convert data into dataframe
df = pd.DataFrame(dataset, columns = feature_names + ["class"])

In [5]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
# There seem to be no null values in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   class              150 non-null    float64
dtypes: float64(5)
memory usage: 6.0 KB


In [7]:
# describe the statistics of individual columns in the data
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [8]:
# print unique values of the target variable, it is descrete as needed for classification
print(df["class"].unique())

[0. 1. 2.]


In [9]:
# The data has no null values and has an equal distribution of each class in the dataset.
# Hence pre-processing of data is not practically needed, we can proceed with test-train split

### test-train split

In [10]:
X = data['data']
y = data['target']

# assign 75% of the data as train, and utilize the rest for test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=23)

### decision-tree implementation

In [11]:
def entropy(y):

    """ The function calculates the entropy based on the formula: -(p_*log(p_) + p*log(p))
            here, p_ = the probability of occurrance of one type of class
                  p = the probability of occurrance of another class type
        """

    # return the count of every unique value in the target label
    _, count = np.unique(y, return_counts=True)
    probabilities = count / len(y)

    return -np.sum([p * np.log2(p) for p in probabilities])

def gain(X, y, feature_index, split_point):

    """ The function returns the information gain based on a split_point for a given feature 
        Formula for information gain, IG = E(Y) - E(Y|X)
    """

    # perform binary split based on the split point of the feature
    left_split = X[:, feature_index] <= split_point
    right_split = X[:, feature_index] > split_point

    # calculate entropy of each split
    left_entropy = entropy(y[left_split])
    right_entropy = entropy(y[right_split])

    # get the weights of split 
    left_weight = np.sum(left_split)/ len(y)
    right_weight = np.sum(right_split)/ len(y)

    # calculate weighted entropy
    weighted_entropy = (left_weight * left_entropy) + (right_weight * right_entropy)

    # return information gain for a specific feature
    return entropy(y) - weighted_entropy

def best_split(X, y):

    """ The function loops over all the unique values in each feature and selects the best feature and split point combination 
        that gives the maximum information gain
    """
    best_feature_index = 0
    best_split_point = 0
    best_gain = -100000

    for index in range(X.shape[1]):
        
        # calculate gain for each feature
        unique_values = np.unique(X[:, index])
        for value in unique_values:
            
            # compare previous gains
            g = gain(X, y, index, value)
            if g > best_gain:
                best_gain = g
                best_feature_index = index
                best_split_point = value

    return best_feature_index, best_split_point

def build_train(X, y, depth=0, max_depth = 3):
    
    """ The function builds a recursive binary decision tree with a maximum allowed depth
        Recursive calls are used to pass the last split data set and build the left and right sub-trees
    """
    # check for the edge case; current_depth > maximum_depth
    if depth >= max_depth:
        return np.bincount(y).argmax()

    # check for edge case; the node has only one type of observations
    if len(np.unique(y)) == 1:
        return y[0]

    # obtain the best split
    best_feature_index, best_split_point = best_split(X, y)

    left_filter = X[:, best_feature_index] <= best_split_point
    right_filter = X[:, best_feature_index] > best_split_point
    
    # recursive calls to build the decision tree
    left_subtree = build_train(X[left_filter], y[left_filter], depth + 1, max_depth)
    right_subtree = build_train(X[right_filter], y[right_filter], depth + 1, max_depth)
    
    return (best_feature_index, best_split_point, left_subtree, right_subtree)

def predict_instance(instance, tree):
    
    """ The function sends individual feature vectors to the model and returns the prediction of the model as an integer
        between 0-2
    """
        
    # edge case; it denotes that we are at the leaf node
    if type(tree) == np.int32 or type(tree) == np.int64:
        return tree
    else:
        feature_index, split_value, left_subtree, right_subtree = tree

        # recursively traverse the tree to check where the corresponding observation falls
        if instance[feature_index] <= split_value:
            return predict_instance(instance, left_subtree)
        else:
            return predict_instance(instance, right_subtree)

def predict(X, tree):
        
        """ The function takes the training set as input and returns the predictions made by the model as an output numoy array
            It calls the predict_instance method on each feature vector of the test set
        """
        
        result = list()
        # loop over the test set and gather predictions
        for instance in X:
            result.append(predict_instance(instance, tree))
        
        return np.array(result)

### model training & evaluation

In [12]:
# build and train the decision tree
tree = build_train(X_train, y_train)

# make predictions using test set
y_pred = predict(X_test, tree)

# evaluate accuracy of the model
accuracy = np.mean(y_pred == y_test)
print("Accuracy of the model is:", accuracy.round(2))

Accuracy of the model is: 0.95


### analysis

### Since the dataset is small, the model is able to perform well and achieves a high accuracy for a tree depth of 3. However, the tree can easily overfit and perform poorly if we slightly increase the depth. 

### Performing only binary splits could potentially limit the performance of decision trees in certain cases. Furthermore the greedy approach followed for splitting might not be the best approach as certain splits might harm the performance than benefit the model.

### However the decision tree formed here is easily interpreted and is non-parametric in nature. So we can be confident about the predictions of our model.